### Initial Set-up of DF and functions, adapted from Saad's

In [1]:
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 280

import matplotlib.pyplot as plt
from collections import defaultdict


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
sw = stopwords.words('english')

In [2]:
dataFolder_path = '../../data/'
data_df = pd.read_csv(dataFolder_path+'judge_1377884607_tweet_product_company.csv')

In [3]:
# def get_wordnet_pos(treebank_tag):
#     '''
#     Translate nltk POS to wordnet tags
#     '''
#     if treebank_tag.startswith('J'):
#         return wordnet.ADJ
#     elif treebank_tag.startswith('V'):
#         return wordnet.VERB
#     elif treebank_tag.startswith('N'):
#         return wordnet.NOUN
#     elif treebank_tag.startswith('R'):
#         return wordnet.ADV
#     else:
#         return wordnet.NOUN


def doc_preparer(doc, stem = False, stop_words=sw):
    '''

    :param doc: a document from the satire corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    #Stemming seems to work better. Lemming can't identify plurals of products
    
    
#     lemmed_keywords = ['apple',
#                 'ipad', 'ipads',
#                 'iphone', 'iphones',
#                 'itunes',
#                 'google', 'googled',
#                 'android', 'droid', 'androids', 'droids',
#                 'circle', 'circles'
#                 'app', 'apps']

#     stemmed_keywords = ['appl',
#                         'ipad',
#                         'iphon',
#                         'itun',
#                         'googl',
#                         'android',
#                         'droid',
#                         'circl',
#                         'app']

    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
#     doc = pos_tag(doc)
#     doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
#     lemmatizer = WordNetLemmatizer()
#     doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
#     doc = [word for word in doc if word in lemmed_keywords]
    
    
    
    p_stemmer = nltk.stem.PorterStemmer()
    if stem:
        doc = [p_stemmer.stem(word) for word in doc if p_stemmer.stem(word)]
    return ' '.join(doc)

def cv_printScores(cv_metric):
    print('CV Results')
    print('='*32)
    print('Accuracy')
    print('-'*32)
    print(f"Training accuracy: {cv_metric['train_accuracy'].mean():.3f}")
    print(f"Test accuracy:     {cv_metric['test_accuracy'].mean():.3f}")
    print('F-1 Score')
    print('-'*32)
    print(f"Training F1 score: {cv_metric['train_f1_macro'].mean():.3f}")
    print(f"Test F1 score:     {cv_metric['test_f1_macro'].mean():.3f}")

In [4]:
# Drop Nulls & I can't tell
data_df.dropna(subset=['tweet_text'],inplace=True)
data_df = data_df[data_df.is_there_an_emotion_directed_at_a_brand_or_product != "I can't tell" ]

In [ ]:
# extend Stop words
sw.extend(['sxsw','rt','quot','austin','sxswi','mention','link'])

In [5]:
# Encode targets
le = LabelEncoder()
data_df['sentiment_target'] = le.fit_transform(data_df.is_there_an_emotion_directed_at_a_brand_or_product)
le.classes_

array(['Negative emotion', 'No emotion toward brand or product',
       'Positive emotion'], dtype=object)

In [8]:
data_df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,sentiment_target
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion,0
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion,2
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion,2
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion,0
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion,2


In [9]:
### Bring in translated DF
neg_tran_df = pd.read_csv('neg_translated.csv')

In [12]:
# Clean up DF from CSV
neg_tran_df.drop('Unnamed: 0', axis=1, inplace=True)
neg_tran_df.set_index('Unnamed: 0.1')

In [16]:
over_neg_df = pd.concat([data_df, neg_tran_df])

In [17]:
over_neg_df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,sentiment_target,Unnamed: 0.1
0,".@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead! I need to upgrade. Plugin stations at #SXSW.",iPhone,Negative emotion,0,NaN
1,"@jessedee Know about @fludapp ? Awesome iPad/iPhone app that you'll likely appreciate for its design. Also, they're giving free Ts at #SXSW",iPad or iPhone App,Positive emotion,2,NaN
2,@swonderlin Can not wait for #iPad 2 also. They should sale them down at #SXSW.,iPad,Positive emotion,2,NaN
3,@sxsw I hope this year's festival isn't as crashy as this year's iPhone app. #sxsw,iPad or iPhone App,Negative emotion,0,NaN
4,"@sxtxstate great stuff on Fri #SXSW: Marissa Mayer (Google), Tim O'Reilly (tech books/conferences) &amp; Matt Mullenweg (Wordpress)",Google,Positive emotion,2,NaN
...,...,...,...,...,...
540,Google Guy from #SXSW Talk explains how he did realistic Twitter bots as an experiment.,NaN,Negative emotion,0,8603.0
541,I think my Effing husband stands in line for a #ipad 2.Can someone point out the line-up for woman number 2? #Sxswi #sxsw,iPad,Negative emotion,0,8611.0
542,"I am pretty sure that the discussion participant who thinks ""Apple drowned in her success"".",Apple,Negative emotion,0,8638.0
543,"Hey, does someone do #SXSW who registers for the GroupMe groups? I got it on my iPhone, but nobody else is there, so ... somehow useless.",NaN,Negative emotion,0,8672.0


In [25]:
# Create stem'd versions
data_df['stemmed_tokens'] = data_df['tweet_text'].map(lambda x:doc_preparer(x,stem=True))
over_neg_df['stemmed_tokens'] = over_neg_df['tweet_text'].map(lambda x:doc_preparer(x,stem=True))

In [26]:
# Not sure how to do an apples to apples comparision without data leakage.  Taking a rough look:
X = data_df['stemmed_tokens']
y = data_df['sentiment_target']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3)

X_neg = over_neg_df['stemmed_tokens']
y_neg = over_neg_df['sentiment_target']

X_neg_train, X_neg_test, y_neg_train, y_neg_test = train_test_split(X_neg, y_neg,test_size = 0.3)

In [27]:
# Default DF
tvec = TfidfVectorizer()

X_train_vec = tvec.fit_transform(X_train)
X_train_vec = pd.DataFrame.sparse.from_spmatrix(X_train_vec)
X_train_vec.columns = sorted(tvec.vocabulary_)
X_train_vec.set_index(y_train.index, inplace=True)

rfc = RandomForestClassifier()

tvec_rfc_cvResults = cross_validate(rfc,
                                    X_train_vec,
                                    y_train,
                                    scoring=('accuracy', 'f1_macro'),
                                    cv=5,
                                    verbose=1,
                                    n_jobs=-2,
                                    return_train_score=True)

cv_printScores(tvec_rfc_cvResults)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   5 | elapsed:   11.2s remaining:   16.9s


CV Results
Accuracy
--------------------------------
Training accuracy: 0.996
Test accuracy:     0.680
F-1 Score
--------------------------------
Training F1 score: 0.997
Test F1 score:     0.495


[Parallel(n_jobs=-2)]: Done   5 out of   5 | elapsed:   12.8s finished


In [28]:
# Test with oversampled neg DF. Same params and setup (will be variation from train/test split)

tvec_neg = TfidfVectorizer()

X_neg_train_vec = tvec_neg.fit_transform(X_neg_train)
X_neg_train_vec = pd.DataFrame.sparse.from_spmatrix(X_neg_train_vec)
X_neg_train_vec.columns = sorted(tvec_neg.vocabulary_)
X_neg_train_vec.set_index(y_neg_train.index, inplace=True)

rfc_neg = RandomForestClassifier()

tvec_neg_rfc_cvResults = cross_validate(rfc_neg,
                                    X_neg_train_vec,
                                    y_neg_train,
                                    scoring=('accuracy', 'f1_macro'),
                                    cv=5,
                                    verbose=1,
                                    n_jobs=-2,
                                    return_train_score=True)

cv_printScores(tvec_neg_rfc_cvResults)

[Parallel(n_jobs=-2)]: Using backend LokyBackend with 11 concurrent workers.
[Parallel(n_jobs=-2)]: Done   2 out of   5 | elapsed:   11.6s remaining:   17.4s


CV Results
Accuracy
--------------------------------
Training accuracy: 0.996
Test accuracy:     0.678
F-1 Score
--------------------------------
Training F1 score: 0.997
Test F1 score:     0.610


[Parallel(n_jobs=-2)]: Done   5 out of   5 | elapsed:   12.9s finished
